# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [25]:
from xml.etree import ElementTree as ET

In [26]:
document = ET.parse( 'mondial_database.xml' )

In [27]:
import pandas as pd

# 1 - 10 countries with the lowest infant mortality rates

In [28]:
# create country and infant mortality lists
ctry = []
mort = []

# build list from xml elements
for element in document.getroot():
    for subelement in element.iter('infant_mortality'):
        ctry.append(element.find('name').text )
        mort.append(float(subelement.text))

# merge lists into a dictionary
ctry_by_mort = {'Country':ctry, 'Infant_mortality':mort}

# create dataframe to sort on mortality rate 
df1 = pd.DataFrame(data=ctry_by_mort)
df1.sort_values('Infant_mortality',ascending=True).head(10)


,Country,Infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# 2 - 10 cities with the largest population

In [29]:
# Create a city list
city = []

# build list from xml elements
for element in document.iterfind('country'):
    for subelement in element.iter('city'):
        # eliminate nulls
        if subelement.find('population') != None: 
            city_by_pop = {}
            city_by_pop['city'] = subelement.find('name').text
            # Take the last one of all populations measured
            city_by_pop['population'] = int(subelement.findall('population')[-1].text)
            city.append(city_by_pop)

# create dataframe to sort on population 
df2 = pd.DataFrame(city)  
df2.sort_values(by = 'population', ascending = False).head(10)


,city,population
1251,Shanghai,22315474
707,Istanbul,13710512
1421,Mumbai,12442373
443,Moskva,11979529
1250,Beijing,11716620
2594,São Paulo,11152344
1252,Tianjin,11090314
974,Guangzhou,11071424
1467,Delhi,11034555
977,Shenzhen,10358381


# 3 - 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [30]:
# create a country list
ctry = []

# build list from xml elements
for element in document.iterfind('country'):
        # read in the data using a datadictionary
        ctry_by_dict = {}
        #make sure there is no null population values
        if element.find('population') != None:
            for ethnic in element.findall('ethnicgroup'):
                ctry_by_dict['country'] = element.find('name').text
                ctry_by_dict['population'] = int(element.findall('population')[-1].text)
                ctry_by_dict['ethnicgroup'] = ethnic.text
                ctry_by_dict['percentage'] = float(ethnic.attrib['percentage'])
                ctry_by_dict['ethnic_pop'] = int(ctry_by_dict['population']*ctry_by_dict['percentage']/100)
                ctry.append(ctry_by_dict)
                #initiate dictionary to capture next ethnicgroup subelement
                ctry_by_dict = {}

# create df for groupby sort
df3 = pd.DataFrame(ctry).drop_duplicates()    
df3.groupby('ethnicgroup').ethnic_pop.sum().sort_values(ascending = False).head(10)


ethnicgroup
Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
Name: ethnic_pop, dtype: int64

# 4 - name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [ ]:
# trying out DOM to create a country_name to country_code map that will be used to merge with other datasets

In [31]:
from xml.dom import minidom
xmldoc = minidom.parse('mondial_database.xml')
mondial = xmldoc.getElementsByTagName("mondial")[0]
countrys = mondial.getElementsByTagName("country")

In [32]:
#create a map list
cty_map = []

#use DOM functions to fill list with xml elements
for country in countrys:
    dict = {}
    dict ['cname']= country_name = country.getElementsByTagName("name")[0].firstChild.data
    dict ['code'] = country_code = country.getAttribute("car_code")
    cty_map.append(dict)

# create a map_df, dropping anu dups         
map_df=pd.DataFrame(cty_map).drop_duplicates()
#print (map_df.head)


# 4a - name and country of a) longest river

In [33]:
#create a river list
rv = []

# build list from xml elements
for element in document.iterfind('river'):
        dict = {}
        # exclude nulls
        if (element.find('length') != None) & (element.find('source') != None): 
            dict['name'] = element.find('name').text
            dict['length'] = float(element.find('length').text)
            dict['code'] = element.find('source').attrib['country']
            rv.append(dict)

# drop missing value          
rv_length=pd.DataFrame(rv).dropna()

# merge with country code/name map for country names
rv_length = rv_length.merge(map_df,on='code')

#rename columns for presentation
rv_length = rv_length.rename(columns={'code':'country_code'})
rv_length = rv_length.rename(columns={'cname':'country'})

#convert lenght to int for presentation
rv_length.length = rv_length.length.map(int)

#sort by length for the longest river in the world
rv_length.sort_values(by = 'length', ascending = False).head(1)


,country_code,length,name,country
171,PE,6448,Amazonas,Peru


# 4b - name and country b) of largest lake

In [34]:
# create a lake list
lake_list = []

# populate list with xml elements
for element in document.iterfind('lake'):
        # read in the data using a data dictionary
        dict = {}
        # check there is no nulls
        if element.find('area') != None: 
            dict['area'] = float(element.find('area').text)
            # take only the first code_id 
            dict['code'] = element.attrib['country'][0]
            dict['name'] = element.find('name').text
            lake_list.append(dict)
        
# create df; drop any nulls              
lakes = pd.DataFrame(lake_list).dropna()  

# merge with country code/name map on code index
lakes = lakes.merge(map_df,on='code')

#rename columns for presentation
lakes = lakes.rename(columns={'code':'country_code'})
lakes = lakes.rename(columns={'cname':'country'})

#convert area to int for presentation
lakes.area = lakes.area.map(int)

#sort by area to get the largest lake in the world
lakes.sort_values(by = 'area', ascending = False).head(1)

,area,country_code,name,country
87,386400,R,Caspian Sea,Russia


# 4c - name and country of airport at highest elevation

In [35]:
# create airport list
air_list = []

# populate list with xml elements
for element in document.iterfind('airport'):
        # read in the data using a data dictionary
        dict = {}
        # check if the elevation is not null
        if element.find('elevation') != None:
            dict['code'] = element.attrib['country']
            dict['name'] = element.find('name').text
            dict['elevation'] = element.find('elevation').text
            air_list.append(dict)
            
# create df; eliminate nulls               
air = pd.DataFrame(air_list).dropna()  

# merge with previous country name/code map on code index
air = air.merge(map_df,on='code')

#convert elevation to int for presentation
air.elevation = air.elevation.map(int)

#rename columns as needed for presentation
air = air.rename(columns={'code':'country_code'})
air = air.rename(columns={'cname':'country'})
air = air.rename(columns={'name':'airport'})

#sort by elevation to get the loftiest airport in the world
air.sort_values(by = 'elevation', ascending = False).head(1)


,country_code,elevation,airport,country
80,BOL,4063,El Alto Intl,Bolivia
